In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import timm
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Identify the number of classes by the number of subfolders in the data directory
NUM_CLASSES = len(os.listdir('./data'))
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [4]:
class TumorClassifier(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(TumorClassifier, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
# Creating model
model = TumorClassifier().to(device)

In [6]:
print('Loading data...')
# Load in the data from the data directory; subdirectories are the classes
data = torchvision.datasets.ImageFolder('./data', transform=torchvision.transforms.ToTensor())

# Split the data into training and validation sets
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size, val_size])

Loading data...


In [7]:
# Create data loaders for the training and validation sets
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Move the model to the GPU
model = model.to(device)

In [8]:
epochs = 26
# Train the model
print('Training model...')
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} loss: {loss.item():.3f}")


Training model...
Epoch 1 loss: 1.238
Epoch 2 loss: 0.985
Epoch 3 loss: 0.657
Epoch 4 loss: 0.627
Epoch 5 loss: 0.641
Epoch 6 loss: 0.552
Epoch 7 loss: 0.196
Epoch 8 loss: 0.599
Epoch 9 loss: 0.289
Epoch 10 loss: 0.179
Epoch 11 loss: 0.083
Epoch 12 loss: 0.552
Epoch 13 loss: 0.233
Epoch 14 loss: 0.151
Epoch 15 loss: 0.109
Epoch 16 loss: 0.486
Epoch 17 loss: 0.223
Epoch 18 loss: 0.464
Epoch 19 loss: 0.093
Epoch 20 loss: 0.247
Epoch 21 loss: 0.099
Epoch 22 loss: 0.256
Epoch 23 loss: 0.367
Epoch 24 loss: 0.107
Epoch 25 loss: 0.016
Epoch 26 loss: 0.003


In [9]:
# Validate the model

print('Validating model...')

correct = 0
total = 0

with torch.no_grad():
    for data in val_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the {val_size} validation images: {(100 * correct / total):.2f}%")



Validating model...
Accuracy of the network on the 620 validation images: 91.61%
